In [1]:
import pandas as pd
import re

In [2]:
raw_data = pd.read_csv('./data/raw_data.csv')

raw_data

,German,French
0,@chardinghh In meiner Branche sind wir damit g...,Du « Pacte des loups » à « Juste la fin du mon...
1,@crittersport 🤣,"Devant le Sénat, Vincent Bolloré relativise so..."
2,Ich vermisse beim Stundeneintragen den Punkt „...,La Corée du Nord annonce une possible reprise ...
3,@lawen4cer N‘Trotzki.,Joe Biden n’avait pas « anticipé » une telle o...
4,@ktschk ER HAT UNS DIE FREUDE GESCHENKT!,"Aux Etats-Unis, Joe Biden promet un « désastre..."
...,...,...
1991,@nikahle @andijah Profi finde ich super!,RT @Sante_Gouv: 💉 Campagne de vaccination cont...
1992,@andijah 😇,"RT @JeanCASTEX: Partout, pour tous, une écolog..."
1993,@andijah Ganz spontan: Guru oder Crack – ist a...,#COVID19 | ✈️ Déplacements à l’international :...
1994,RT @andijah: Kurze Zwischenfrage ins virtuelle...,RT @Interieur_Gouv: 📽️ Après avoir quitté le V...


## Some ideas:
- `@` + word + ` ` 
- Lowercase everything

### Remove
- Numbers
- Hashtags
- Links
- Special characters such as *:, (, ), etc.*
- Emojis
- `"RT"`

### Special characters
- \ - " "( )  / : _ * $ + 

In [3]:
languages = raw_data.columns
languages

Index(['German', 'French'], dtype='object')

In [4]:
def clean_data(text):
    # Hashtag filter
    no_hashtag = text.apply(lambda x: re.sub('#[\w]+','', x)) 

    # Mention filter
    no_mention = no_hashtag.apply(lambda x: re.sub("@([A-Za-z0-9_ßäöüÄÖÜÇâêîôûéàèùëïç]+)|([^0-9A-Za-z_ßäöüÄÖÜâêîôûéàèùëïç \t])|(\w+:\/\/\S+)",' ', x))

    # Remove RT
    no_rt = no_mention.apply(lambda x: re.sub('RT','', x))

    # Remove numbers
    no_num = no_rt.apply(lambda x: re.sub('[0-9]+','', x))

    # Remove extra underscores
    no_underscore = no_num.apply(lambda x: re.sub('_','', x))

    # Remove leading and trailing whitespaces
    no_space = no_underscore.apply(lambda x: x.strip())

    # Lowercase the words
    lowercase = no_space.apply(lambda x: x.lower())

    # Remove extra spaces in between words
    cleaned = lowercase.apply(lambda x: " ".join(x.split()))

    return cleaned

In [5]:
cleaned_df = pd.DataFrame()

for language in languages:
    sentences = raw_data[language].dropna(axis=0)
    cleaned_df[language] = clean_data(sentences)

In [6]:
cleaned_df

,German,French
0,in meiner branche sind wir damit ganz weit vorn,du pacte des loups à juste la fin du monde la ...
1,,devant le sénat vincent bolloré relativise son...
2,ich vermisse beim stundeneintragen den punkt e...,la corée du nord annonce une possible reprise ...
3,n trotzki,joe biden n avait pas anticipé une telle oppos...
4,er hat uns die freude geschenkt,aux etats unis joe biden promet un désastre à ...
...,...,...
1991,profi finde ich super,campagne de vaccination contre le au janvier p...
1992,,partout pour tous une écologie du concret qui ...
1993,ganz spontan guru oder crack ist aber beides a...,déplacements à l international l afrique du su...
1994,kurze zwischenfrage ins virtuelle großraumbüro...,après avoir quitté le vietnam à ans vy est arr...


In [7]:
cleaned_df.to_csv('./data/cleaned_data.csv', index=False)